In [1]:
# Boiler plate stuff to start the module
import jpype
import jpype.imports
from jpype.types import *
import sys 
import numpy as np
import traceback

In [2]:
# Launch the JVM
jpype.startJVM(classpath=['./maze-server-v2019.4.jar'])

In [3]:
# import the Java modules
from java.io import IOException
from java.net import Socket
from java.net import UnknownHostException

from javax.net.ssl import SSLSocket
from javax.net.ssl import SSLSocketFactory
from javax.xml.bind import JAXBException
from javax.xml.bind import UnmarshalException

from de.fhac.mazenet.server.generated import AwaitMoveMessageData
from de.fhac.mazenet.server.generated import Errortype
from de.fhac.mazenet.server.generated import MazeCom
from de.fhac.mazenet.server.generated import MazeComMessagetype
from de.fhac.mazenet.server.generated import MoveMessageData
from de.fhac.mazenet.server.generated import BoardData

from de.fhac.mazenet.server.networking import MazeComMessageFactory
from de.fhac.mazenet.server.networking import XmlInputStream
from de.fhac.mazenet.server.networking import XmlOutputStream
from de.fhac.mazenet.server.game import Board
from de.fhac.mazenet.server.game import Position

In [4]:
class Client:
    __id = None
    def __init__(self, name, address, port, tls):
        if(tls):
            self.sslSocketFactory = SSLSocketFactory.getDefault()
            self.sslSocket = self.sslSocketFactory.createSocket(address, port)
            self.in_ = XmlInputStream(self.sslSocket.getInputStream())
            self.out = XmlOutputStream(self.sslSocket.getOutputStream())
        else:
            self.socket = Socket(address, port)
            self.in_ = XmlInputStream(self.socket.getInputStream())
            self.out = XmlOutputStream(self.socket.getOutputStream())

        self.name = name
        self.instance = None
    
    def getId(self):
        return self.__id
    
    def resetId(self):
        self.__id = 0

    @staticmethod
    def getInstance(self, name, address, port, tls):
        if(self.instance == None): self.instance = Client(name, address, port, tls)
        return self.instance

    def login(self):
        login = MazeComMessageFactory.createLoginMessage(self.name)
        self.out.write(login)
        loginResponse = self.in_.readMazeCom()
        if(loginResponse.getMessagetype() == MazeComMessagetype.LOGINREPLY):
            self.__id = loginResponse.getLoginReplyMessage().getNewID()
        elif(loginResponse.getMessagetype() == MazeComMessagetype.ACCEPT):
            print("Login Fehler: Falsche Nachricht")
            sys.exit(1)
        elif (loginResponse.getMessagetype() == MazeComMessagetype.DISCONNECT):
            print("Login Fehler: Loginversuche")
            sys.exit(1)
        else:
            print("Login Fehler: bekommte Message Type" + loginResponse.getMessagetype())
            sys.exit(1)

    def play(self):
        while(True):
            try:
                receivedMazeCom = self.in_.readMazeCom()
                if(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                    self.awaitMove(receivedMazeCom)
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                    self.accept(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                    print("You have been disconnected")
                    self.disconnect(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                    print("in MoveInfo")
                elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                    print("You have won")
                    sys.exit(0)
                else:
                    print("Unkown message type: " + receivedMazeCom.getMessagetype())
            except Exception:
                print(traceback.format_exc())

    def accept(self, errortype):
        if(errortype ==  Errortype.NOERROR):
            print("Alles Ok, Bewegung erlaubt.")
        elif(errortype ==  Errortype.AWAIT_MOVE):
            print("Falsche Nachricht, Bewegung nicht erlaubt.")
        elif(errortype ==  Errortype.ILLEGAL_MOVE):
            print("Regelverstoß, unerlaubte Bewegung.")
        else:
            print("Unbekannte Fehlermeldung: " + errortype)
    
    def disconnect(self, errortype):
        if(errortype ==  Errortype.TOO_MANY_TRIES):
            print("Zu viele Versuche. Geworfen vom Server.")
            sys.exit(1)
        else:
            print("Ende des Spiels.")
            sys.exit(0)

    def awaitMove(self, receivedMazeCom):
        awaitMove = receivedMazeCom.getAwaitMoveMessage()
        allPossibleBoards = []
        if(awaitMove != None):
            boardData = awaitMove.getBoard()
            board = Board(boardData)
            potentialShiftMoves = []
            for i in np.arange(1, 6, 2):
                potentialShiftMoves.append(Position(0, i))
                potentialShiftMoves.append(Position(6, i))
                potentialShiftMoves.append(Position(i, 0))
                potentialShiftMoves.append(Position(i, 6))
            
            potentialShiftMoves.remove(board.getForbidden())

            move = MoveMessageData()
            move.setShiftCard(boardData.getShiftCard())
            move.setNewPinPos(board.findPlayer(self.__id))
            move.setShiftPosition(potentialShiftMoves[0])
            
            mazeComToSend = MazeCom()
            mazeComToSend.setId(self.__id)
            mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
            mazeComToSend.setMoveMessage(move)
            self.out.write(mazeComToSend)

In [5]:
HOST = "127.0.0.1"
PORT = 5123

In [6]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [7]:
from mss import mss
import pydirectinput
import cv2
import pytesseract
import matplotlib.pyplot as plt
import time

In [8]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [9]:
class MazeEnv(Env):
    def __init__(self):
        super().__init__()
        # Actions we can take, rotate card left, rotate card right, push card, go up, down, left, right, no op
        # actions: push card, move, no op
        # self.action_space = Discrete(3)
        # actions: pick x and y where the next card should be pushed
        self.action_space = Box(low=np.array([0,0]), high=np.array([6,6]), dtype=np.int8)
        #  array
        #self.observation_space = Tuple((
        #                             Discrete(50),
        #                             Discrete(24),
        #                             Discrete(2))
        #                            )
        self.observation_space = Box(low=0, high=255, shape=(1,800,1100), dtype=np.uint8)
        self.cap = mss()
        self.game_location = {'top':200, 'left':100, 'width':1700, 'height':800}
        self.end_location = {'top':50, 'left':400, 'width':400, 'height':150}
        self.client = 0
    """    
        self.players = np.array([0,0,0,0])
        
    def add_player(self, client):
        for i in range(4):
            if self.players[i] == 0:
                self.players[i] = client
                break
    """    
    def step(self, action):
        reward = 0
        #if self.client.getId() == 0:
        #    self.client.login()
        #    time.sleep(1000)
        try:
            receivedMazeCom = self.client.in_.readMazeCom()
            if(receivedMazeCom.getMessagetype() == MazeComMessagetype.AWAITMOVE):
                awaitMove = self.client.in_.readMazeCom().getAwaitMoveMessage()
                if(awaitMove != None):
                    boardData = awaitMove.getBoard()
                    board = Board(boardData)

                    move = MoveMessageData()
                    move.setShiftCard(boardData.getShiftCard())
                    move.setNewPinPos(board.findPlayer(client.getId()))
                    move.setShiftPosition(Position(action[0], action[1]))

                    mazeComToSend = MazeCom()
                    mazeComToSend.setId(self.__id)
                    mazeComToSend.setMessagetype(MazeComMessagetype.MOVE)
                    mazeComToSend.setMoveMessage(move)
                    self.client.out.write(mazeComToSend)

                    reward += 10
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.ACCEPT):
                self.client.accept(receivedMazeCom.getAcceptMessage().getErrortypeCode())
                print("aaccepted")
                reward += 5
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.DISCONNECT):
                print("You have been disconnected")
                reward -= 10
                self.client.disconnect(receivedMazeCom.getDisconnectMessage().getErrortypeCode())
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.MOVEINFO):
                print("in MoveInfo")
            elif(receivedMazeCom.getMessagetype() == MazeComMessagetype.WIN):
                print("You have won")
                reward += 100
                sys.exit(0)
            else:
                print("Unkown message type: " + receivedMazeCom.getMessagetype())
        except Exception:
            print(traceback.format_exc())
        #############
        done, done_cap = self.get_done() 
        observation = self.get_observation()
        info = {}
        return observation, reward, done, info
        
    def render(self):
        # Implement viz
        plt.imshow(np.array(self.cap.grab(self.end_location)))
    
    def reset(self):
        time.sleep(1)
        # stop
        pydirectinput.click(x=1550, y=200)
        # start
        pydirectinput.click(x=1450, y=200)
        print("before sleep")
        time.sleep(10)
        print("after sleeeeeeep")
        #self.client = Client(self.client.name, HOST, PORT, False)
        self.client.login()
        return self.get_observation()
    
    # Closes the game?
    def close(self):
        pass
    
    def get_observation(self):
        raw = np.array(self.cap.grab(self.game_location))[:,:,:3]
        
        gray = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY) 
        
        resized = cv2.resize(gray, (100,83))
        channel = np.reshape(resized, (1,83,100))
        return channel
    
    def get_done(self):
        done_screen = np.array(self.cap.grab(self.end_location))
        
        done_strs = ['Winner', 'Winn']
        
        # ocr
        done = False
        psm = r'--psm 8'
        res = pytesseract.image_to_string(done_screen, config=psm)[:4]
        if res in done_strs:
            done = True
        
        return done, done_screen

In [14]:
menv = MazeEnv()

In [17]:
menv.client = Client('joe', HOST, PORT, False)

In [ ]:
obs = menv.reset()
done = False  
total_reward   = 0
while not done: 
    obs, reward,  done, info =  menv.step(menv.action_space.sample())
    print('before sleep in looooop')
    time.sleep(10)
    print('after sleep in looooop')
    total_reward  += reward
print('Total Reward for episode {} is {}'.format(episode, total_reward))

before sleep
after sleeeeeeep
before sleep in looooop
after sleep in looooop
Traceback (most recent call last):
  File "XmlInputStream.java", line 68, in de.fhac.mazenet.server.networking.XmlInputStream.readMazeCom
  File "UTFInputStream.java", line 20, in de.fhac.mazenet.server.networking.UTFInputStream.readUTF8
  File "UTFInputStream.java", line 41, in de.fhac.mazenet.server.networking.UTFInputStream.readNBytes
Exception: Java Exception

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\AStA\AppData\Local\Temp\ipykernel_12592\444313929.py", line 35, in step
    receivedMazeCom = self.client.in_.readMazeCom()
java.io.java.io.EOFException: java.io.EOFException: [Fehler]: Konnte nur 0 von 4 angefordeten Bytes lesen

before sleep in looooop
after sleep in looooop
Traceback (most recent call last):
  File "XmlInputStream.java", line 68, in de.fhac.mazenet.server.networking.XmlInputStream.readMazeCom
  File "UTFInputSt

before sleep in looooop
after sleep in looooop
Traceback (most recent call last):
  File "XmlInputStream.java", line 68, in de.fhac.mazenet.server.networking.XmlInputStream.readMazeCom
  File "UTFInputStream.java", line 20, in de.fhac.mazenet.server.networking.UTFInputStream.readUTF8
  File "UTFInputStream.java", line 41, in de.fhac.mazenet.server.networking.UTFInputStream.readNBytes
Exception: Java Exception

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\AStA\AppData\Local\Temp\ipykernel_12592\444313929.py", line 35, in step
    receivedMazeCom = self.client.in_.readMazeCom()
java.io.java.io.EOFException: java.io.EOFException: [Fehler]: Konnte nur 0 von 4 angefordeten Bytes lesen

before sleep in looooop
after sleep in looooop
Traceback (most recent call last):
  File "XmlInputStream.java", line 68, in de.fhac.mazenet.server.networking.XmlInputStream.readMazeCom
  File "UTFInputStream.java", line 20, in de.fha

In [ ]:
s

In [ ]:
try:
    c = Client('joe', HOST, PORT, False)
    c.login()
    c.play()
except Exception:
    print(traceback.format_exc())

In [ ]:
# jpype.shutdownJVM()